While I'm waiting for a full parmaeter cube to be generated, I'm testing the emulation framework I've made so far. /

In [33]:
import numpy as np
from src.emulator import build_emulator, emulate, emulate_wrt_r, get_plot_data
from src.allCorrFunc import RBINS
from src.doBatchCalls import BOUNDS, N_PER_DIM

In [34]:
for key, val in BOUNDS.iteritems():
    print key
    print np.linspace(val[0], val[1], num=N_PER_DIM)
    print '*'*30

logMmin
[ 11.7         11.96666667  12.23333333  12.5       ]
******************************
f_c
[ 0.1         0.23333333  0.36666667  0.5       ]
******************************
logM0
[ 10.  11.  12.  13.]
******************************
sigma_logM
[ 0.2         0.36666667  0.53333333  0.7       ]
******************************
logM1
[ 13.1  13.5  13.9  14.3]
******************************
alpha
[ 0.75        0.91666667  1.08333333  1.25      ]
******************************


In [35]:
bias = True
y_param = 'logMmin'
#y_param = 'f_c'
#y_param = 'logM0'
#y_param = 'logM1'
#y_param = 'sigma_logM'
#y_param = 'alpha'

#param_2 = 'logMmin'
param_2 = 'sigma_logM'
#param_2 = 'f_c'
#parma_2 = 'logM0'

#param_3 = 'sigma_logM'
param_3 = 'logM0'
#param_3='f_c'

#param_4 = 'alpha'
#param_4 = 'logM1'
param_4 = 'f_c'

#param_5= 'logM1'
#param_5 = 'f_c'
#param_5='alpha'

emulation_point = [('f_c', 0.233), 
                   ('logM0', 12.0), ('sigma_logM', 0.533), ('alpha', 1.083),
                   ('logM1', 13.5), ('logMmin', 12.233)]

fiducial_point = {'logM0': 12.20, 'logM1': 13.7, 'alpha': 1.02, 
                  'logMmin': 12.1, 'f_c': 0.19, 'sigma_logM': 0.46}
fixed_params = {key:val for key, val in emulation_point}
#em_params = {}
em_params = {param_2:fixed_params[param_2], param_3:fixed_params[param_3]}#,
            #param_4:fixed_params[param_4]}#, param_5:fixed_params[param_5]} #varying y_param, holding this fixed in the emulator now. 
del fixed_params[y_param]
del fixed_params[param_2]
del fixed_params[param_3]
#del fixed_params[param_4]
#del fixed_params[param_5]

In [36]:
#%%timeit
from time import time
t0 = time()
gp, log_xi, log_xi_cov = build_emulator(fixed_params, bias=bias)
print '%.3f s'%(time()-t0)

64 Points used for training.
Initial Params:  [ 6.    0.03  0.03  0.03  0.03]
GP Optimization Failed.
GP Params:  [ 6.72734705  0.05172751  0.02408372  0.03052224  0.03282459]
68.623 s


In [60]:
from scipy.linalg import det, eigh, inv
print log_xi_cov[0,0]
print det(log_xi_cov/log_xi_cov[0,0])

0.000128864952226
2.73742847668e-201


In [62]:
D, P = eigh(log_xi_cov[:18, :][:,:18])
test = P.conj()*D*P
print P.T*P

[[  9.39709880e-06  -6.04959108e-06   7.35786164e-07  -2.17095950e-04
   -7.29778729e-04  -3.97028930e-04   1.33477516e-03  -4.99852371e-04
   -3.01573628e-03  -4.47839351e-03  -3.28409826e-03  -1.54229950e-02
   -1.03450485e-02  -2.55574906e-03  -7.97565464e-02  -5.29880124e-04
   -2.34686235e-04  -9.72217086e-06]
 [ -6.04959108e-06   9.15165229e-04  -1.36647765e-04   1.62095372e-04
    3.92864509e-05  -1.15762248e-03  -6.46280566e-04   1.32804448e-03
   -7.48135481e-03   2.96756087e-02  -3.76017622e-02  -3.76802053e-03
    1.61984507e-04  -9.29382769e-03   1.94153759e-03  -2.48612546e-03
   -2.87139515e-04  -1.29229883e-06]
 [  7.35786164e-07  -1.36647765e-04   1.47594106e-03  -4.36474852e-05
   -6.53748119e-04  -3.30346694e-03   2.36618850e-06   1.57839895e-02
    1.12800616e-02   6.88555125e-05  -1.02181083e-01  -2.66931711e-01
    4.05801036e-03   1.92296898e-03  -2.16290945e-03   1.90798660e-02
   -5.98079299e-03  -4.76553034e-05]
 [ -2.17095950e-04   1.62095372e-04  -4.36474852e

In [ ]:
from scipy.linalg import eigvals
c = log_xi_cov[:18,:][:,:18]
for e in eigvals(c):
    assert e > 0

In [ ]:
log_xi_mean = log_xi.mean()
log_xi-=log_xi_mean

In [ ]:
#gp.kernel[:] = [0.175, -0.32, -3.4]

In [ ]:
yp = np.linspace(BOUNDS[y_param][0],BOUNDS[y_param][1], num=N_PER_DIM)
N = 200
rpoints = np.linspace(-1, 1.5, N)
RBIN_CENTERS = ((RBINS[:-1]+RBINS[1:])/2)
outputs = emulate_wrt_r(gp, log_xi, em_params, rpoints,
                                    y_param=y_param,y_points=yp)

In [ ]:
from matplotlib import pyplot as plt
from itertools import cycle
%matplotlib inline
#plt.style.use('fivethirtyeight')
import seaborn as sns
sns.set()
colors = cycle(sns.color_palette())

In [ ]:
print log_xi.shape

In [ ]:
plot_r, plot_xi, plot_xi_err = get_plot_data(em_params, fixed_params, bias=bias)

In [ ]:
fig = plt.figure(figsize = (10, 10))
for label, (log_xi_pred, log_xi_err),pr, px, err,c in zip(yp, outputs,plot_r, plot_xi,plot_xi_err,colors):

    plt.plot(10**rpoints, (log_xi_pred+log_xi_mean),label='%.3f'%label,color = c)
    #plt.fill_between(10**rpoints,10**(log_xi_pred+log_xi_mean+log_xi_err),
    #                 10**(log_xi_pred+log_xi_mean-log_xi_err), color = c, alpha = 0.3)
    
    plt.fill_between(10**rpoints,log_xi_pred+log_xi_mean+log_xi_err,
                     log_xi_pred+log_xi_mean-log_xi_err, color = c, alpha = 0.3)

    plt.errorbar(10**pr, px,err, color =c,fmt = 'o')

plt.xlim(xmin = 0.1, xmax = 30)    
if bias:
    plt.ylim(ymin = 0, ymax = 15)
else:
    plt.ylim(ymin = -2, ymax = 4)
plt.title("Emulator w.r.t. %s"%y_param, fontsize =20)
#plt.loglog()
plt.xscale('log')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel(r'$r $  $\rm{[Mpc]}$', fontsize=25)
if bias:
    plt.ylabel(r'$\xi_{\rm gg}/\xi_{\rm mm} }$', fontsize=25)
else:
    plt.ylabel(r'$\log_{10}{\xi_{\rm gg}(r) }$', fontsize=25)
plt.legend(loc='best', fontsize=20)  

In [ ]:
N = 200
plot_xi = plot_xi.T #switch rows and cols
plot_xi_err = plot_xi_err.T
plot_r = plot_r.T
yp = np.linspace(BOUNDS[y_param][0],BOUNDS[y_param][1], num=N)
plot_yp = np.linspace(BOUNDS[y_param][0],BOUNDS[y_param][1], num=N_PER_DIM)
rpoints = np.log10(RBIN_CENTERS)
#print rpoints
outputs = emulate(gp, log_xi, em_params,x_param=y_param, x_points=yp,y_param='r',y_points=rpoints)

In [ ]:
i = 4
#remove some points
rpoints=rpoints[::i]
outputs=outputs[::i]
plot_r=plot_r[::i]
plot_xi=plot_xi[::i]
plot_xi_err = plot_xi_err[::i]

In [ ]:
fig = plt.figure(figsize = (10, 10))

for label, (log_xi_pred, log_xi_err),px,err,c in zip(rpoints, outputs, plot_xi,plot_xi_err,colors):

    plt.plot(yp, (log_xi_pred+log_xi_mean),label='%.3f Mpc'%(10**label),color = c)
    
    plt.fill_between(yp,log_xi_pred+log_xi_mean+log_xi_err,
                     log_xi_pred+log_xi_mean-log_xi_err, color = c, alpha = 0.3)
    plt.errorbar(plot_yp, px,err, color =c,fmt = 'o')
    
plt.xlim(xmin = 0.97*BOUNDS[y_param][0], xmax = BOUNDS[y_param][1]*1.03)
if bias:
    plt.ylim(ymin=0, ymax=15)
else:
    plt.ylim(ymin = -2.5, ymax = 3.5)
plt.title("Emulator w.r.t. %s"%y_param, fontsize =20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
#plt.yscale('log')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel(r'$\log_{10}{M_{min}}}$  $\rm{[Log_{10} M_{\odot}]}$', fontsize=25)
#plt.xlabel(r'$\alpha$ ', fontsize=25)
if bias:
    plt.ylabel(r'$\xi_{\rm gg}/\xi_{\rm mm} }$', fontsize=25)
else:
    plt.ylabel(r'$\log_{10}{\xi_{\rm gg}(r) }$', fontsize=25)
plt.legend(loc='best', fontsize=20)  